In [1]:
import pandas as pd
import numpy as np

# Procurando um curso prático de Data Science que cabe no seu bolso?
# CursoDeDataScience.com
# Começa 20:05 - não pule!

In [87]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train['Sex_binario'] = train['Sex'].map({"male": 0, "female": 1})

test['Sex_binario'] = test['Sex'].map({"male": 0, "female": 1})

In [88]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_binario
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [82]:
X = train.select_dtypes(include=np.number).drop(["PassengerId", 'Survived'], axis=1).fillna(0)
y = train['Survived']

In [78]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.pipeline import make_pipeline

In [89]:
kf = KFold(n_splits=2, random_state=0, shuffle=True)


second_level = np.zeros((X.shape[0], 4))

for tr, ts in kf.split(X,y):
    Xtr, Xval = X.iloc[tr], X.iloc[ts]
    ytr, yval = y.iloc[tr], y.iloc[ts]
    
    rf = RandomForestClassifier(n_estimators=100, n_jobs=6, random_state=10)
    rf.fit(Xtr, ytr)
    prf = rf.predict_proba(Xval)[:,1]
    prf_ = (prf > 0.5).astype(int)
    
    print("RF Accuracy: {} - Log Loss: {}".format(accuracy_score(yval, prf_), log_loss(yval, prf)))
    
    et = ExtraTreesClassifier(n_estimators=100, n_jobs=6, random_state=10)
    et.fit(Xtr, ytr)
    pet = et.predict_proba(Xval)[:,1]
    pet_ = (pet > 0.5).astype(int)
    
    print("ET Accuracy: {} - Log Loss: {}".format(accuracy_score(yval, pet_), log_loss(yval, pet)))
    
    lr1 = make_pipeline(StandardScaler(), LogisticRegression())
    lr1.fit(Xtr, ytr)
    plr1 = lr1.predict_proba(Xval)[:,1]
    plr1_ = (plr1 > 0.5).astype(int)
    
    print("LR StdScaler Accuracy: {} - Log Loss: {}".format(accuracy_score(yval, plr1_), log_loss(yval, plr1)))
    
    lr2 = make_pipeline(MinMaxScaler(), LogisticRegression())
    lr2.fit(Xtr, ytr)
    plr2 = lr2.predict_proba(Xval)[:,1]
    plr2_ = (plr2 > 0.5).astype(int)
    
    print("LR MinMax Accuracy: {} - Log Loss: {}".format(accuracy_score(yval, plr2_), log_loss(yval, plr2)))
    
    second_level[ts, 0] = prf
    second_level[ts, 1] = pet
    second_level[ts, 2] = plr1
    second_level[ts, 3] = plr2
    
    print()
    
# fatores de diversidade

RF Accuracy: 0.7869955156950673 - Log Loss: 0.8891877224352214
ET Accuracy: 0.7802690582959642 - Log Loss: 1.7277767849421113
LR StdScaler Accuracy: 0.7713004484304933 - Log Loss: 0.4548870233895985
LR MinMax Accuracy: 0.773542600896861 - Log Loss: 0.45730271249069515

RF Accuracy: 0.8292134831460675 - Log Loss: 0.6313360340802087
ET Accuracy: 0.8089887640449438 - Log Loss: 2.533916920090966
LR StdScaler Accuracy: 0.7955056179775281 - Log Loss: 0.4579224442106097
LR MinMax Accuracy: 0.802247191011236 - Log Loss: 0.4610789905245168



In [ ]:
# second_level.mean(axis=1)

In [ ]:
folds = [1, 2, 3]

primeiro nivel

ciclo 1 = [1,2] [3]
ciclo 2 = [1,3] [2]
ciclo 3 = [2,3] [1]

segundo nivel

ciclo 1 = [1,2] [3]
ciclo 2 = [1,3] [2]
ciclo 3 = [2,3] [1]






In [ ]:
# modelos - gbm + rede neural, knn 
# features
# exemplos 
# hiperparametros 

In [97]:
second_level

array([[0.15      , 0.19      , 0.0947167 , 0.11582552],
       [0.95      , 0.99      , 0.90010202, 0.88974478],
       [0.54      , 0.76      , 0.65737866, 0.63540321],
       ...,
       [0.74      , 0.76      , 0.61083421, 0.59159169],
       [0.84      , 0.92      , 0.45575654, 0.45086285],
       [0.35583333, 0.4       , 0.10479333, 0.11872796]])

In [95]:
for tr, ts in kf.split(X,y):
    
    Xtr, Xval = second_level[tr], second_level[ts]
    ytr, yval = y.iloc[tr], y.iloc[ts]
    
    lr_stack = LogisticRegression(C=1.)
    lr_stack.fit(Xtr, ytr)
    plr_stack = lr_stack.predict_proba(Xval)[:,1]
    plr_stack_ = (plr_stack > 0.5).astype(int)
    
    print("Stack Accuracy: {}  Log loss: {}".format(accuracy_score(yval, plr_stack_), log_loss(yval, plr_stack)))
    print()

Stack Accuracy: 0.8004484304932735  Log loss: 0.4379152913562436

Stack Accuracy: 0.8157303370786517  Log loss: 0.42892608158477763



In [75]:
pd.DataFrame(np.corrcoef(second_level.T))

,0,1,2,3
0,1.000000,0.935380,0.595684,0.578839
1,0.935380,1.000000,0.497313,0.479914
2,0.595684,0.497313,1.000000,0.987037
3,0.578839,0.479914,0.987037,1.000000
